# Details on ChaoticNDE

In the main notebook we make use of `ChaoticNDE` from `ChaoticNDETools`. Here, are some details on the implementation of it, to make the Neural DE model easily trainable with the training data we have set up.

`ChaoticNDE` is set up similar to how other neural network layers and models are set up in the `Flux.jl` library. We define a `struct` (similar to classes in OOP-languages like Python) that holds all information about the Neural DE:
* its parameters `p`
* `prob`, the `ODEProblem` that defines the initial value problem to solve for the ODE solver
* the solver algorithm that we want to `alg`
    * if none is given `Tsit5()` is used, a Runge-Kutta 4/5 solver with adaptive stepsize 
* any additional keyword arguments for the solver 
    * this could be for example `reltol` for controlling the accuracy (of the adaptive stepping) 
    * we can also specifiy the exact algorithm we want to use for computing the gradients here via the `sensealg` keyword
    
We make this type parameteric with the `{P,R,A,K}` parameters. They are stand-ins for any concrete type that we initialize the `struct` with when running the code. This construction lets the compiler optimize the code better. 

In [ ]:
abstract type AbstractChaoticNDEModel end 

"""
    ChaoticNDE{P,R,A,K} <: AbstractChaoticNDEModel

Model for setting up and training Chaotic Neural Differential Equations.

# Fields:

* `p` parameter vector 
* `prob` DEProblem 
* `alg` Algorithm to use for the `solve` command 
* `kwargs` any additional keyword arguments that should be handed over (e.g. `sensealg`)

# Constructor 

`ChaoticNDE(prob; alg=Tsit5(), kwargs...)`
"""
struct ChaoticNDE{P,R,A,K} <: AbstractChaoticNDEModel
    p::P 
    prob::R 
    alg::A
    kwargs::K
end 

Then, we define an alternative way to initialize this struct, just based of a previously defined `ODEProblem`: 

In [ ]:
function ChaoticNDE(prob; alg=Tsit5(), kwargs...)
    p = prob.p 
    ChaoticNDE{typeof(p), typeof(prob), typeof(alg), typeof(kwargs)}(p, prob, alg, kwargs)
end 

Next, we tell `Flux.jl` that this is a trainable model with its parameters being the field `p` of the struct:

In [ ]:
Flux.@functor ChaoticNDE
Flux.trainable(m::ChaoticNDE) = (p=m.p,)

Last thing we do is to overload the struct, so that we can call the struct like a function after we initialized it:

In [ ]:
function (m::ChaoticNDE)(X,p=m.p)
    (t, x) = X 
    Array(solve(remake(m.prob; tspan=(t[1],t[end]),u0=x[:,1],p=p), m.alg; saveat=t, m.kwargs...))
end

This allow a syntax like this: 

In [ ]:
model = ChaoticNDE(prob) # this code will not run in this notebook
model(input)

In the [actual package](https://github.com/maximilian-gelbrecht/ChaoticNDETools.jl/blob/main/src/models.jl) we also do some additional bookkeeping to make the Neural DE also runnable on GPUs and load parameters. 